In [1]:
#第6章/加载tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('hfl/rbt3')

tokenizer

PreTrainedTokenizerFast(name_or_path='hfl/rbt3', vocab_size=21128, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [2]:
#第6章/试编码句子
tokenizer.batch_encode_plus(
    ['明月装饰了你的窗子', '你装饰了别人的梦'],
    truncation=True,
)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[101, 3209, 3299, 6163, 7652, 749, 872, 4638, 4970, 2094, 102], [101, 872, 6163, 7652, 749, 1166, 782, 4638, 3457, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [3]:
#第6章/从磁盘加载数据集
from datasets import load_from_disk

dataset = load_from_disk('./data/ChnSentiCorp')

#缩小数据规模，便于测试
dataset['train'] = dataset['train'].shuffle().select(range(2000))
dataset['test'] = dataset['test'].shuffle().select(range(100))

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 0
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
})

In [4]:
#第6章/编码
def f(data):
    return tokenizer.batch_encode_plus(data['text'], truncation=True)


dataset = dataset.map(f,
                      batched=True,
                      batch_size=1000,
                      num_proc=4,
                      remove_columns=['text'])

dataset

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['label'],
        num_rows: 0
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 100
    })
})

In [5]:
#第6章/移除太长的句子
def f(data):
    return [len(i) <= 512 for i in data['input_ids']]


dataset = dataset.filter(f, batched=True, batch_size=1000, num_proc=4)

dataset

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1980
    })
    validation: Dataset({
        features: ['label'],
        num_rows: 0
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 99
    })
})

In [6]:
#第6章/加载模型
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('hfl/rbt3',
                                                           num_labels=2)

#统计模型参数量
sum([i.nelement() for i in model.parameters()]) / 10000

Some weights of the model checkpoint at hfl/rbt3 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3

3847.8338

In [7]:
#第6章/模型试算
import torch

#模拟一批数据
data = {
    'input_ids': torch.ones(4, 10, dtype=torch.long),
    'token_type_ids': torch.ones(4, 10, dtype=torch.long),
    'attention_mask': torch.ones(4, 10, dtype=torch.long),
    'labels': torch.ones(4, dtype=torch.long)
}

#模型试算
out = model(**data)

out['loss'], out['logits'].shape

(tensor(0.4744, grad_fn=<NllLossBackward>), torch.Size([4, 2]))

In [8]:
# #第6章/加载评价指标
# from datasets import load_metric

# metric = load_metric('accuracy')

In [9]:
#第6章/定义评价函数
import numpy as np
from transformers.trainer_utils import EvalPrediction


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = logits.argmax(axis=1)
    return {'accuracy': (logits == labels).sum() / len(labels)}
    #return metric.compute(predictions=logits, references=labels)


#模拟输出
eval_pred = EvalPrediction(
    predictions=np.array([[0, 1], [2, 3], [4, 5], [6, 7]]),
    label_ids=np.array([1, 1, 0, 1]),
)

compute_metrics(eval_pred)

{'accuracy': 0.75}

In [10]:
#第6章/定义训练参数
from transformers import TrainingArguments

#定义训练参数
args = TrainingArguments(
    #定义临时数据保存路径
    output_dir='./output_dir',

    #定义测试执行的策略，可取值no、epoch、steps
    evaluation_strategy='steps',

    #定义每隔多少个step执行一次测试
    eval_steps=30,

    #定义模型保存策略，可取值no、epoch、steps
    save_strategy='steps',

    #定义每隔多少个step保存一次
    save_steps=30,

    #定义共训练几个轮次
    num_train_epochs=1,

    #定义学习率
    learning_rate=1e-4,

    #加入参数权重衰减，防止过拟合
    weight_decay=1e-2,

    #定义测试和训练时的批次大小
    per_device_eval_batch_size=16,
    per_device_train_batch_size=16,

    #定义是否要使用gpu训练
    no_cuda=False,
)

In [11]:
#第6章/定义训练器
from transformers import Trainer
from transformers.data.data_collator import DataCollatorWithPadding

#定义训练器
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer),
)

In [12]:
#第6章/测试数据整理函数
data_collator = DataCollatorWithPadding(tokenizer)

#获取一批数据
data = dataset['train'][:5]

#输出这些句子的长度
for i in data['input_ids']:
    print(len(i))

#调用数据整理函数
data = data_collator(data)

#查看整理后的数据
for k, v in data.items():
    print(k, v.shape)

71
53
75
42
181
input_ids torch.Size([5, 181])
token_type_ids torch.Size([5, 181])
attention_mask torch.Size([5, 181])
labels torch.Size([5])


In [13]:
tokenizer.decode(data['input_ids'][0])

'[CLS] 收 到 书 之 后 第 一 感 觉 就 是 脏 脏 的, 随 手 翻 了 几 页, 发 现 里 面 竟 有 破 损 的 纸 张, 我 倒! 有 的 页 面 上 还 有 漆 黑 的 脚 印, 当 当 的 工 作 人 员 发 书 时 难 道 不 确 认 么??? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [14]:
#第6章/评价模型
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 99
  Batch size = 16


{'eval_loss': 0.716060996055603,
 'eval_accuracy': 0.42424242424242425,
 'eval_runtime': 0.5896,
 'eval_samples_per_second': 167.899,
 'eval_steps_per_second': 11.872}

In [15]:
#第6章/训练
trainer.train()

/root/anaconda3/envs/gpu/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1980
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 124


Step,Training Loss,Validation Loss,Accuracy
30,No log,0.368228,0.838384
60,No log,0.342529,0.868687
90,No log,0.452716,0.858586
120,No log,0.307068,0.898990


***** Running Evaluation *****
  Num examples = 99
  Batch size = 16
Saving model checkpoint to ./output_dir/checkpoint-30
Configuration saved in ./output_dir/checkpoint-30/config.json
Model weights saved in ./output_dir/checkpoint-30/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 99
  Batch size = 16
Saving model checkpoint to ./output_dir/checkpoint-60
Configuration saved in ./output_dir/checkpoint-60/config.json
Model weights saved in ./output_dir/checkpoint-60/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 99
  Batch size = 16
Saving model checkpoint to ./output_dir/checkpoint-90
Configuration saved in ./output_dir/checkpoint-90/config.json
Model weights saved in ./output_dir/checkpoint-90/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 99
  Batch size = 16
Saving model checkpoint to ./output_dir/checkpoint-120
Configuration saved in ./output_dir/checkpoint-120/config.json
Model weights saved in ./output_dir/checkpoint-120/pyt

TrainOutput(global_step=124, training_loss=0.39254542320005353, metrics={'train_runtime': 45.7338, 'train_samples_per_second': 43.294, 'train_steps_per_second': 2.711, 'total_flos': 70209086949120.0, 'train_loss': 0.39254542320005353, 'epoch': 1.0})

In [16]:
#第6章/从某个存档继续训练
trainer.train(resume_from_checkpoint='./output_dir/checkpoint-90')

Loading model from ./output_dir/checkpoint-90).
***** Running training *****
  Num examples = 1980
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 124
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 0
  Continuing training from global step 90
  Will skip the first 0 epochs then the first 90 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/90 [00:00<?, ?it/s]

Step,Training Loss,Validation Loss,Accuracy
120,No log,0.359170,0.868687


***** Running Evaluation *****
  Num examples = 99
  Batch size = 16
Saving model checkpoint to ./output_dir/checkpoint-120
Configuration saved in ./output_dir/checkpoint-120/config.json
Model weights saved in ./output_dir/checkpoint-120/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=124, training_loss=0.07213338728873961, metrics={'train_runtime': 13.1996, 'train_samples_per_second': 150.004, 'train_steps_per_second': 9.394, 'total_flos': 68529398541984.0, 'train_loss': 0.07213338728873961, 'epoch': 1.0})

In [17]:
#第6章/评价模型
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 99
  Batch size = 16


{'eval_loss': 0.36910974979400635,
 'eval_accuracy': 0.8484848484848485,
 'eval_runtime': 0.5673,
 'eval_samples_per_second': 174.517,
 'eval_steps_per_second': 12.34,
 'epoch': 1.0}

In [18]:
#第6章/手动保存模型参数
trainer.save_model(output_dir='./output_dir/save_model')

Saving model checkpoint to ./output_dir/save_model
Configuration saved in ./output_dir/save_model/config.json
Model weights saved in ./output_dir/save_model/pytorch_model.bin


In [19]:
#第6章/手动加载模型参数
import torch

model.load_state_dict(torch.load('./output_dir/save_model/pytorch_model.bin'))

<All keys matched successfully>

In [20]:
#第6章/测试
model.eval()

for i, data in enumerate(trainer.get_eval_dataloader()):
    break

for k, v in data.items():
    data[k] = v.to('cuda')

out = model(**data)
out = out['logits'].argmax(dim=1)

for i in range(16):
    print(tokenizer.decode(data['input_ids'][i], skip_special_tokens=True))
    print('label=', data['labels'][i].item())
    print('predict=', out[i].item())

原 3255 ， 300 卷 + 瑞 星 2009.. 实 际 价 : 2995. 比 较 实 惠 。 屏 比 较 大 哦 。 独 显 不 错 ， 一 般 够 用 了 。
label= 1
predict= 1
服 务 很 差 ！ ！ ！ 服 务 员 的 态 度 很 不 好 ！ 环 境 也 一 般
label= 0
predict= 0
不 足 就 是 没 有 配 套 光 盘 ， 需 要 自 己 备 份 。 我 不 太 喜 欢 这 种 触 摸 板 ， 没 有 送 鼠 标 ， 还 要 自 己 去 配 ， 这 个 都 是 小 问 题 。 无 线 网 卡 插 槽 的 按 钮 太 突 出 了 ， 很 怕 不 小 心 会 弄 断 。 不 过 使 用 不 要 那 么 野 蛮 呵 呵 就 不 会 有 大 问 题 。
label= 0
predict= 0
设 施 很 不 错 ； 周 边 环 境 不 错 ， 紧 邻 金 沙 滩 ； 推 荐 海 景 房 不 足 ： 比 较 偏 ； 吃 饭 不 放 便 ； 酒 店 自 带 的 餐 厅 性 价 比 太 差 。 价 格 太 贵 ， 口 味 不 好 ；
label= 1
predict= 1
大 家 都 说 这 本 书 写 了 很 多 心 理 案 例, 但 我 觉 得 什 么 案 例 呀 ， 不 就 是 作 者 编 的 故 事 么 ， 而 且 欠 分 析 。 说 什 么 是 心 理 学 的 辅 读 读 物 ， 我 看 就 是 把 一 些 故 事 串 起 来 ， 再 打 乱 一 下 顺 序 而 已 。 推 荐 那 些 喜 欢 看 小 说 的 人 看 ， 但 真 想 学 到 点 心 理 学 知 识 ， 那 就 免 谈 了 。
label= 0
predict= 0
贾 志 刚 是 有 才 的 ， 所 以 写 了 春 秋 ， 但 人 嘛 就 向 春 秋 那 样 乱 ， 不 对 ， 说 错 了 ！ 应 该 说 ， 人 心 嘛 ， 就 像 春 秋 那 样 乱 。 所 以 《 春 秋 》 一 出 ， 赞 弹 皆 有 ， 其 实 贾 志 刚 说 得 对 ， 不 看 春 秋 ， 连 祖 宗 姓 什 么 都 不 知 道 呢 ！ 虽 然 书 中 句 子 略 有 不 雅 ， 就 是 俗 了 点 ， 但 俗 容 易 看 明 白 。 如 果 写 书 的 都 整 点 文